# Домашнее задание к лекции "Базовые понятия статистики"

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. 

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

### Задание 3. Работа с пропусками

Рассчитать количество выбросов(**Я посчитал, что тут опечатка и нужно посчитать количество пропусков. Если нет - все выбросы выведены и посчитаны в пункте 2**) для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

## Дополнительная часть (необязательная)

Выполнить задания 1-3 для всего набора данных.

## Решение


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('horse_data.csv', header = None, usecols = [0,1,3,4,10,12,21,22], 
                 names = ['surgery', 'age', 'rect_temp', 'pulse', 'pain', 'abd_dist','protein','outcome' ],
                na_values= '?')

df.head()

,surgery,age,rect_temp,pulse,pain,abd_dist,protein,outcome
0,2.0,1,38.5,66.0,5.0,4.0,NaN,2.0
1,1.0,1,39.2,88.0,3.0,2.0,2.0,3.0
2,2.0,1,38.3,40.0,3.0,1.0,NaN,1.0
3,1.0,9,39.1,164.0,2.0,4.0,5.3,2.0
4,2.0,1,37.3,104.0,NaN,NaN,NaN,2.0


## Задание 1

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   surgery    299 non-null    float64
 1   age        300 non-null    int64  
 2   rect_temp  240 non-null    float64
 3   pulse      276 non-null    float64
 4   pain       245 non-null    float64
 5   abd_dist   244 non-null    float64
 6   protein    102 non-null    float64
 7   outcome    299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


In [4]:
df.describe()

,surgery,age,rect_temp,pulse,pain,abd_dist,protein,outcome
count,299.000000,300.000000,240.000000,276.000000,245.00000,244.000000,102.000000,299.000000
mean,1.397993,1.640000,38.167917,71.913043,2.95102,2.266393,3.019608,1.551839
std,0.490305,2.173972,0.732289,28.630557,1.30794,1.065131,1.968567,0.737187
min,1.000000,1.000000,35.400000,30.000000,1.00000,1.000000,0.100000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,2.00000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,3.00000,2.000000,2.250000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,4.00000,3.000000,3.900000,2.000000
max,2.000000,9.000000,40.800000,184.000000,5.00000,4.000000,10.100000,3.000000


Выводы:
1) surgery - Примерно у 60% лошадей была операция
2) age - максимальное значение 9 при допустимых 1 и 2 - явно проблемы с данными
3) rect_temp - среднее значение примерно равно медиане - это значит, что распределение примерно симметрично относительно нормального значения (37,8 согласно документации)
4) pulse - сверху явно есть выбросы
5) pain - нельзя использовать как численную переменную (согласно документации), но исходя из примерного равенства среднего и медианы, а также велиины стандартного отклонения (почти 1,25), кажется что лошади примерно равномерно распределены по 5 категориям
6) abd_dist - выводы, примерно похожие на предыдущий пункт. Кажется, что между этим показателем и болью есть некая корреляция (что также упомянуто в документации)
7) protein - отсутствуют примерно 2/3 значений, не лучший выбор для анализа
8) outcome - по крайней мере половина лошадей выжила

## Задание 2

 **Surgery**

Для surgery выбросов нет, т.к. max = 2, min =1

**Age**

In [5]:
df[df.age >1].age

3      9
9      9
13     9
16     9
23     9
39     9
41     9
55     9
74     9
75     9
82     9
103    9
109    9
135    9
191    9
212    9
227    9
229    9
244    9
252    9
255    9
262    9
275    9
287    9
Name: age, dtype: int64

Согласно документации, поле age принимает значения 1 и 2, а мы имеем 1 и 9. Мне кажется, здесь ошибка в данных и 9 нужно перекодировать в 2

**Rec_temp**

In [6]:
q1 = df.rect_temp.quantile(0.25)
q3 = df.rect_temp.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
keep_outliers = df[(df.rect_temp<lower_bound) | (df.rect_temp>upper_bound)].rect_temp.sort_values()
print(lower_bound, upper_bound)
keep_outliers

36.74999999999999 39.550000000000004


44     35.4
141    36.0
238    36.1
80     36.4
118    36.5
298    36.5
251    36.6
99     39.6
75     39.7
20     39.9
281    40.0
54     40.3
91     40.3
259    40.8
Name: rect_temp, dtype: float64

То, что попало в выбросы ведёт себя достаточно плавно и не сильно выбивается из интервалов, которые мы наметили как границы для выбросов. Предлагаю интерпретировать эти данные как обычные нормальные цифры.

**pulse**

In [7]:
q1 = df.pulse.quantile(0.25)
q3 = df.pulse.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
keep_outliers = df[(df.pulse<lower_bound) | (df.pulse>upper_bound)].pulse.sort_values()
print(lower_bound, upper_bound)
keep_outliers

-12.0 148.0


41     150.0
275    150.0
55     160.0
3      164.0
255    184.0
Name: pulse, dtype: float64

Хочется выкинуть из рассмотрения значение 184 - очень сильно выбивается даже из остальных выбросов. Все остальные воспринимать как нормальные значения.

**pain**

Категориальная переменная с минимальным значением 1 и максимальным 5, оба значения валидные и есть в документации. Для этой переменной нет выбросов.

**abd_dist**

Аналогично предыдущему пункту, выбросов нет.

**protein**

In [8]:
q1 = df.protein.quantile(0.25)
q3 = df.protein.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
keep_outliers = df[(df.protein<lower_bound) | (df.protein>upper_bound)].protein.sort_values()
print(lower_bound, upper_bound)
keep_outliers

-0.8499999999999996 6.75


71      7.0
143     7.0
284     7.0
31      7.4
225     8.0
113    10.0
211    10.1
Name: protein, dtype: float64

Значения 10 и 10.1 кажутся сильно выбивающимися и их хочется выкинуть из рассмотрения, но поскольку сама переменная не заполнена в 2/3 случаев, я бы ничего выкидывать не стал (возможно отсутствие такого большого числа данных как раз нарушают равномерность распределения)

**outcome**

Категориальная переменная с минимальным значением 1 и максимальным 3, оба значения валидные и есть в документации. Для этой переменной нет выбросов.

## Задание 3

In [9]:
df.describe()

,surgery,age,rect_temp,pulse,pain,abd_dist,protein,outcome
count,299.000000,300.000000,240.000000,276.000000,245.00000,244.000000,102.000000,299.000000
mean,1.397993,1.640000,38.167917,71.913043,2.95102,2.266393,3.019608,1.551839
std,0.490305,2.173972,0.732289,28.630557,1.30794,1.065131,1.968567,0.737187
min,1.000000,1.000000,35.400000,30.000000,1.00000,1.000000,0.100000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,2.00000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,3.00000,2.000000,2.250000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,4.00000,3.000000,3.900000,2.000000
max,2.000000,9.000000,40.800000,184.000000,5.00000,4.000000,10.100000,3.000000


**Surgery**

Категориальная переменная с двумя значениями, всего один пропуск. Думаю, нормально будет заполнить более частым значением, то есть единицей

In [10]:
df['surgery'] = df['surgery'].fillna(1)

In [11]:
df.describe()

,surgery,age,rect_temp,pulse,pain,abd_dist,protein,outcome
count,300.000000,300.000000,240.000000,276.000000,245.00000,244.000000,102.000000,299.000000
mean,1.396667,1.640000,38.167917,71.913043,2.95102,2.266393,3.019608,1.551839
std,0.490023,2.173972,0.732289,28.630557,1.30794,1.065131,1.968567,0.737187
min,1.000000,1.000000,35.400000,30.000000,1.00000,1.000000,0.100000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,2.00000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,3.00000,2.000000,2.250000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,4.00000,3.000000,3.900000,2.000000
max,2.000000,9.000000,40.800000,184.000000,5.00000,4.000000,10.100000,3.000000


**Age**

Пропусков нет, всё ок

**Outcome**

Случай, аналогичный surgery, заполняем единственный пропуск самым частым значением - 1

In [12]:
df['outcome'] = df['outcome'].fillna(1)

In [13]:
df.describe()

,surgery,age,rect_temp,pulse,pain,abd_dist,protein,outcome
count,300.000000,300.000000,240.000000,276.000000,245.00000,244.000000,102.000000,300.000000
mean,1.396667,1.640000,38.167917,71.913043,2.95102,2.266393,3.019608,1.550000
std,0.490023,2.173972,0.732289,28.630557,1.30794,1.065131,1.968567,0.736642
min,1.000000,1.000000,35.400000,30.000000,1.00000,1.000000,0.100000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,2.00000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,3.00000,2.000000,2.250000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,4.00000,3.000000,3.900000,2.000000
max,2.000000,9.000000,40.800000,184.000000,5.00000,4.000000,10.100000,3.000000


**Rect_temp**

In [14]:
df[df.rect_temp.isna()]

,surgery,age,rect_temp,pulse,pain,abd_dist,protein,outcome
5,2.0,1,NaN,NaN,2.0,2.0,NaN,1.0
7,1.0,1,NaN,60.0,NaN,2.0,NaN,2.0
8,2.0,1,NaN,80.0,4.0,4.0,NaN,3.0
16,1.0,9,NaN,128.0,4.0,3.0,4.7,2.0
28,1.0,1,NaN,NaN,NaN,NaN,NaN,2.0
34,1.0,1,NaN,100.0,5.0,4.0,NaN,1.0
35,2.0,1,NaN,104.0,4.0,3.0,NaN,3.0
40,1.0,1,NaN,88.0,5.0,3.0,NaN,2.0
43,1.0,1,NaN,120.0,4.0,4.0,2.0,3.0
45,2.0,1,NaN,120.0,5.0,4.0,NaN,2.0


Явной коррелляции с другими переменными для пустых значений rect_temp я не вижу.
Сама переменная непрерывная, при этом явных выбросов по ней нет (все выбросы по этой переменной были признаны адекватными в предыдущем пункте задания), поэтому можно и хочется взять среднее значение.
Плюс у нас две изначально заполенные переменные - surgery и outcome, в которых все группы по значениям более-менее многочисленны.
Поэтому заполню пустые значения (60 штук - много, но терпимо) средними для пары surgery и outcome. Ну и по логике кажется, что температура вполне может корреллировать как с наличием операции, так и с летальным исходом.

In [15]:
fill_mean_by_groups = df
fill_mean_by_groups.rect_temp.fillna(fill_mean_by_groups.groupby(['surgery', 'outcome']).rect_temp.transform('mean'), inplace=True)

fill_mean_by_groups.groupby(['surgery', 'outcome']).rect_temp.mean()

surgery  outcome
1.0      1.0        38.144444
         2.0        38.187179
         3.0        38.052381
2.0      1.0        38.240278
         2.0        38.157143
         3.0        38.053846
Name: rect_temp, dtype: float64

На самом деле все значения получились близкими, но перестраховались и перестраховались - не жалко

In [17]:
df = fill_mean_by_groups
df.describe()

,surgery,age,rect_temp,pulse,pain,abd_dist,protein,outcome
count,300.000000,300.000000,300.000000,276.000000,245.00000,244.000000,102.000000,300.000000
mean,1.396667,1.640000,38.166605,71.913043,2.95102,2.266393,3.019608,1.550000
std,0.490023,2.173972,0.655229,28.630557,1.30794,1.065131,1.968567,0.736642
min,1.000000,1.000000,35.400000,30.000000,1.00000,1.000000,0.100000,1.000000
25%,1.000000,1.000000,37.900000,48.000000,2.00000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.187179,64.000000,3.00000,2.000000,2.250000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,4.00000,3.000000,3.900000,2.000000
max,2.000000,9.000000,40.800000,184.000000,5.00000,4.000000,10.100000,3.000000


**Pulse**

In [16]:
df[df.pulse.isna()]

,surgery,age,rect_temp,pulse,pain,abd_dist,protein,outcome
5,2.0,1,38.240278,NaN,2.0,2.0,NaN,1.0
28,1.0,1,38.187179,NaN,NaN,NaN,NaN,2.0
52,2.0,1,38.240278,NaN,1.0,1.0,NaN,1.0
56,1.0,1,38.144444,NaN,NaN,NaN,NaN,1.0
58,1.0,1,38.187179,NaN,5.0,3.0,NaN,2.0
74,1.0,9,38.187179,NaN,NaN,NaN,NaN,2.0
78,1.0,1,38.187179,NaN,5.0,3.0,NaN,2.0
83,1.0,1,38.000000,NaN,5.0,4.0,NaN,2.0
93,2.0,1,38.157143,NaN,5.0,3.0,NaN,2.0
115,2.0,1,38.240278,NaN,3.0,2.0,NaN,1.0


Очень похоже на ситуацию с переменной rect_temp - явных корреляций с другими переменными нет, сама переменная непрерывная, пустых значений 24 - много, но не критично, поэтому заполнять будем средними в группах по surgery и outcome. Правда тут у нас был один странный выброс - значение 184, поэтому его при рассчёте средних учитывать не будем.

In [21]:
fill_mean_by_groups = df
fill_mean_by_groups.pulse.fillna(fill_mean_by_groups[fill_mean_by_groups.pulse != 184].groupby(['surgery', 'outcome']).pulse.transform('mean'), inplace=True)

fill_mean_by_groups.groupby(['surgery', 'outcome']).pulse.mean()

surgery  outcome
1.0      1.0        68.483146
         2.0        87.873922
         3.0        80.037037
2.0      1.0        57.448718
         2.0        84.470588
         3.0        84.625000
Name: pulse, dtype: float64

Здесь средние в разных группах выглядят совсем по-разному, поэтому такой вариант заполнения кажется удачным

In [22]:
df = fill_mean_by_groups
df.describe()

,surgery,age,rect_temp,pulse,pain,abd_dist,protein,outcome
count,300.000000,300.000000,300.000000,300.000000,245.00000,244.000000,102.000000,300.000000
mean,1.396667,1.640000,38.166605,72.146264,2.95102,2.266393,3.019608,1.550000
std,0.490023,2.173972,0.655229,27.665153,1.30794,1.065131,1.968567,0.736642
min,1.000000,1.000000,35.400000,30.000000,1.00000,1.000000,0.100000,1.000000
25%,1.000000,1.000000,37.900000,48.000000,2.00000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.187179,66.000000,3.00000,2.000000,2.250000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,4.00000,3.000000,3.900000,2.000000
max,2.000000,9.000000,40.800000,184.000000,5.00000,4.000000,10.100000,3.000000


**Pain**

In [23]:
df[df.pain.isna()]

,surgery,age,rect_temp,pulse,pain,abd_dist,protein,outcome
4,2.0,1,37.300000,104.000000,NaN,NaN,NaN,2.0
7,1.0,1,38.187179,60.000000,NaN,2.0,NaN,2.0
17,2.0,1,37.500000,48.000000,NaN,NaN,NaN,1.0
19,2.0,1,39.400000,110.000000,NaN,3.0,NaN,1.0
24,1.0,1,38.100000,60.000000,NaN,3.0,NaN,1.0
25,2.0,1,37.800000,60.000000,NaN,NaN,NaN,1.0
27,1.0,1,37.800000,48.000000,NaN,2.0,1.3,1.0
28,1.0,1,38.187179,86.187500,NaN,NaN,NaN,2.0
36,2.0,1,38.300000,112.000000,NaN,1.0,1.0,3.0
39,1.0,9,39.200000,146.000000,NaN,NaN,NaN,2.0


In [37]:
df.groupby(['surgery', 'outcome']).pain.apply(lambda x: x.mode()[0])

surgery  outcome
1.0      1.0        3.0
         2.0        5.0
         3.0        2.0
2.0      1.0        1.0
         2.0        5.0
         3.0        3.0
Name: pain, dtype: float64

In [ ]:
Переменная категориальная, явных коррелляций нет, заполним 55 пропусков модой по группам surgery и outcome

In [39]:
fill_mean_by_groups = df
fill_mean_by_groups.pain.fillna(fill_mean_by_groups.groupby(['surgery', 'outcome']).pain.transform(lambda x: x.mode()[0]), inplace=True)

surgery  outcome
1.0      1.0        3.0
         2.0        5.0
         3.0        2.0
2.0      1.0        1.0
         2.0        5.0
         3.0        3.0
Name: pain, dtype: float64

In [40]:
df = fill_mean_by_groups
df.describe()

,surgery,age,rect_temp,pulse,pain,abd_dist,protein,outcome
count,300.000000,300.000000,300.000000,300.000000,300.000000,244.000000,102.000000,300.000000
mean,1.396667,1.640000,38.166605,72.146264,2.953625,2.266393,3.019608,1.550000
std,0.490023,2.173972,0.655229,27.665153,1.209909,1.065131,1.968567,0.736642
min,1.000000,1.000000,35.400000,30.000000,1.000000,1.000000,0.100000,1.000000
25%,1.000000,1.000000,37.900000,48.000000,2.000000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.187179,66.000000,3.000000,2.000000,2.250000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,4.000000,3.000000,3.900000,2.000000
max,2.000000,9.000000,40.800000,184.000000,5.000000,4.000000,10.100000,3.000000


**Abd_dist**

In [41]:
df[df.abd_dist.isna()]

,surgery,age,rect_temp,pulse,pain,abd_dist,protein,outcome
4,2.0,1,37.300000,104.000000,3.529412,NaN,NaN,2.0
17,2.0,1,37.500000,48.000000,2.072464,NaN,NaN,1.0
23,1.0,9,38.300000,130.000000,2.000000,NaN,NaN,1.0
25,2.0,1,37.800000,60.000000,2.072464,NaN,NaN,1.0
28,1.0,1,38.187179,86.187500,3.791667,NaN,NaN,2.0
33,1.0,1,38.200000,64.000000,3.000000,NaN,6.6,1.0
39,1.0,9,39.200000,146.000000,3.791667,NaN,NaN,2.0
41,2.0,9,39.000000,150.000000,2.072464,NaN,0.1,1.0
44,1.0,1,35.400000,140.000000,4.000000,NaN,2.0,3.0
53,2.0,1,38.600000,40.000000,2.072464,NaN,NaN,1.0


In [42]:
df.groupby(['surgery', 'outcome']).abd_dist.apply(lambda x: x.mode()[0])

surgery  outcome
1.0      1.0        1.0
         2.0        3.0
         3.0        4.0
2.0      1.0        1.0
         2.0        4.0
         3.0        3.0
Name: abd_dist, dtype: float64

Ситуация и решение абсолютно аналогичны переменной pain - заменяем модами по группам surgery и outcome

In [44]:
fill_mean_by_groups = df
fill_mean_by_groups.abd_dist.fillna(fill_mean_by_groups.groupby(['surgery', 'outcome']).abd_dist.transform(lambda x: x.mode()[0]), inplace=True)


In [45]:
df = fill_mean_by_groups
df.describe()

,surgery,age,rect_temp,pulse,pain,abd_dist,protein,outcome
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,102.000000,300.000000
mean,1.396667,1.640000,38.166605,72.146264,2.953625,2.236667,3.019608,1.550000
std,0.490023,2.173972,0.655229,27.665153,1.209909,1.109697,1.968567,0.736642
min,1.000000,1.000000,35.400000,30.000000,1.000000,1.000000,0.100000,1.000000
25%,1.000000,1.000000,37.900000,48.000000,2.000000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.187179,66.000000,3.000000,2.000000,2.250000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,4.000000,3.000000,3.900000,2.000000
max,2.000000,9.000000,40.800000,184.000000,5.000000,4.000000,10.100000,3.000000


**protein**

Пропущенных значений слишком много - 198, хочется исключить столбец из датафрейма


In [48]:
df = df.drop(columns = ['protein'])

In [50]:
df.describe()

,surgery,age,rect_temp,pulse,pain,abd_dist,outcome
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,1.396667,1.640000,38.166605,72.146264,2.953625,2.236667,1.550000
std,0.490023,2.173972,0.655229,27.665153,1.209909,1.109697,0.736642
min,1.000000,1.000000,35.400000,30.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.900000,48.000000,2.000000,1.000000,1.000000
50%,1.000000,1.000000,38.187179,66.000000,3.000000,2.000000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,4.000000,3.000000,2.000000
max,2.000000,9.000000,40.800000,184.000000,5.000000,4.000000,3.000000


Теперь в df лежит датафрейм без пустых значений.